In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from datetime import datetime
from selenium.webdriver.support.ui import Select
from pathlib import Path
import json

def img(newspaper, date, page, datafile='data.json'):
    current_data = json.loads(Path(datafile).read_text())
    year = str(date.year)
    month = str(date.month)
    day = str(date.day)
    page = str(page)
    try:
        return current_data[newspaper][year][month][day][page]
    except:
        return False
    
def login(b):
    b.get('https://www.newspapers.com/signin/')
    sleep(2)
    b.find_element_by_id('username').send_keys('') #add username
    sleep(1)
    b.find_element_by_id('password').send_keys('') #add password
    b.find_element_by_id('password').send_keys(Keys.ENTER)
    sleep(4)

In [2]:
try:
    b
except:
    b = webdriver.Firefox()
    login(b)

In [3]:
datafile = 'data.json'

In [20]:
# Search parameter
searches = [
    {
        'newspaper': 'Albion News',
        'date': datetime(1932, 2, 4),
        'page': 4
    },
    {
        'newspaper': 'Albion News',
        'date': datetime(1932, 2, 4),
        'page': 3
    }
]

In [26]:
for search in searches:
    newspaper, date, page = search['newspaper'], search['date'], search['page']

    if not img(newspaper, date, page):
        print('--> SEARCHING correct newspaper')
        b.get('https://www.newspapers.com/papers/#')
        b.find_element_by_id('filter-text').send_keys(newspaper, Keys.ENTER)
        sleep(2)

        print('   --> FINDING ALL results')
        print(len(b.find_elements_by_class_name('record-content')), 'results found')
        actual_results = []
        for result in b.find_elements_by_class_name('record-content'):
            if result.find_element_by_tag_name('h2').text.lower() == newspaper.lower():
                actual_results.append(result)

        print('   --> DISAMBIGUATING results')
        if len(actual_results) > 1:
            print('ambivalent results found. help!')
            actual_results = []
        else:
            actual_results = actual_results[0]
            link = actual_results.find_element_by_tag_name('a').get_attribute('href')

        sleep(2)
        print(f'   --> QUERYING url ({link})')
        b.get(link)
        sleep(4)

        print('   --> SELECTING correct year')
        b.find_element_by_id('datepicker_year_combobox').send_keys(date.year, Keys.ENTER)
        sleep(2)

        print('   --> SELECTING correct month')
        month_selector = Select(b.find_element_by_id('select_month'))
        month_selector.select_by_visible_text(date.strftime('%B'))
        sleep(2)

        print('   --> FINDING relevant day in calendar')
        relevant_day = [x for x in b.find_elements_by_class_name('hasmatches') if x.get_attribute('href').split('#')[1] == date.strftime("%Y-%m-%d")]
        if len(relevant_day) > 1:
            print('found more than one relevant day... weird!')
            relevant_day = []
        elif len(relevant_day) == 1:
            relevant_day = relevant_day[0]
            url = relevant_day.get_attribute('href')
        else:
            print('found no relevant days :(')

        relevant_day.click()
        sleep(6)

        print('   --> SELECTING correct page')
        b.find_element_by_id('filmstrip_pagenum_target_input').clear()
        sleep(1)
        b.find_element_by_id('filmstrip_pagenum_target_input').send_keys(page)
        sleep(1)
        b.find_element_by_id('filmstrip_pagenum_target_input').send_keys(Keys.ENTER)

        print('   --> GETTING correct img_id')
        img_id = [x for x in b.current_url.split('/') if x and not x in ['https:', 'www.newspapers.com', 'image']][0]

        print(f'   --> WRITING img_id to datafile ({datafile})')
        if not Path(datafile).exists():
            Path(datafile).write_text(json.dumps({}))

        current_data = json.loads(Path(datafile).read_text())

        if not newspaper in current_data:
            current_data[newspaper] = {}

        if not str(date.year) in current_data[newspaper]:
            current_data[newspaper][str(date.year)] = {}

        if not str(date.month) in current_data[newspaper][str(date.year)]:
            current_data[newspaper][str(date.year)][str(date.month)] = {}

        if not str(date.day) in current_data[newspaper][str(date.year)][str(date.month)]:
            current_data[newspaper][str(date.year)][str(date.month)][str(date.day)] = {}

        if not str(page) in current_data[newspaper][str(date.year)][str(date.month)][str(date.day)]:
            current_data[newspaper][str(date.year)][str(date.month)][str(date.day)][str(page)] = img_id

        Path(datafile).write_text(json.dumps(current_data))
    else:
        print(f'found `{newspaper}, {date.strftime("%Y-%m-%d")}, {page}`: ', img(newspaper, date, page))

found `Albion News, 1932-02-04, 4`:  671554386
--> SEARCHING correct newspaper
   --> FINDING ALL results
2 results found
   --> DISAMBIGUATING results
   --> QUERYING url (https://www.newspapers.com/paper/albion-news/23933/)
   --> SELECTING correct year
   --> SELECTING correct month
   --> FINDING relevant day in calendar
   --> SELECTING correct page
   --> GETTING correct img_id
   --> WRITING img_id to datafile (data.json)


[]